# Ratterdam Jupyter Notebook for Classical Statistics
## ANOVAs, ...

In [1]:
import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import numpy as np
from scipy.stats import sem, binom_test, shapiro, bartlett
import utility_fx as util
import os
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import ratterdam_DataFiltering as Filt
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from scipy.stats import kruskal

In [2]:
%qtconsole --style native

### Beltway Rate Remapping 2-factor ANOVA
#### factor 1: Texture (3 levels: A, B, C); factor 2: position (n levels, each spatial bin per alley)
#### This as of 4/19 is considering a 2 way anova on each alley separately and using a Bonferroni correction to take into account multiple alleys

In [5]:
datafile = "E:\\Ratterdam\\R781\\Beltway_D3_190307\\"
expCode = "BRD3"

In [13]:
datafile = "E:\\Ratterdam\\R781\\Beltway_D3_190307\\"
expCode = "BRD3"
alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)

In [10]:
nbins = Def.singleAlleyBins[0]-1
alphaCorr = 0.05/((9+3+30)) # nominal 0.05 alpha level downward corrected via Bonferroni for number of alleys tested

In [8]:
alley=1
posLevels, txtLevels, firingRates = [], [], []

for visit in unit.alleys[alley]:
    frs = visit['ratemap1d']
    txt = visit['metadata']['stimulus']
    
    firingRates.extend(frs)
    posLevels.extend(list(range(nbins)))
    txtLevels.extend([txt]*nbins)

In [9]:
dataDict = {'Position': posLevels, 'Texture': txtLevels, 'FiringRate': firingRates}
df = pandas.DataFrame(dataDict, columns = ['Position', 'Texture', 'FiringRate'])

In [50]:
model = ols('FiringRate ~ C(Position)*C(Texture)', df).fit()

In [11]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             FiringRate   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     2.183
Date:                Mon, 15 Apr 2019   Prob (F-statistic):           1.46e-07
Time:                        16:18:24   Log-Likelihood:                -1542.9
No. Observations:                 510   AIC:                             3264.
Df Residuals:                     421   BIC:                             3641.
Df Model:                          88                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.2815      2.074      0.136      0.892      -3.794       4.357
C(Position)[T.1]                     -0.2793      3.786     -0.074      0.941      -7.721       7.162
C(Position)[T.2]                      0.7763      2.932      0.265      0.791      -4.988       6.540
C(Position)[T.3]                     -0.2607      3.212     -0.081      0.935      -6.575       6.053
C(Position)[T.4]                      2.0778      3.052      0.681      0.496      -3.922       8.077
C(Position)[T.5]                     -0.1697      3.052     -0.056      0.956      -6.169       5.830
C(Position)[T.6]                      2.1051      3.052      0.690      0.491      -3.894       8.104
C(Position)[T.7]                      2.2045      3.052      0.722      0.471      -3.795       8.204
C(Position)[T.8]                      2.5545      3.212      0.795      0.427      -3.760       8.869
C(Position)[T.9]                      1.4638      2.839      0.516      0.606      -4.117       7.045
C(Position)[T.10]                     1.5063      2.765      0.545      0.586      -3.928       6.941
C(Position)[T.11]                     1.2088      2.932      0.412      0.680      -4.555       6.973
C(Position)[T.12]                     0.8578      2.839      0.302      0.763      -4.723       6.439
C(Position)[T.13]                    -0.0462      2.932     -0.016      0.987      -5.810       5.718
C(Position)[T.14]                     3.7531      2.932      1.280      0.201      -2.011       9.517
C(Position)[T.15]                    -0.1522      3.052     -0.050      0.960      -6.152       5.847
C(Position)[T.16]                     7.0431      2.932      2.402      0.017       1.279      12.807
C(Position)[T.17]                     0.7872      3.212      0.245      0.807      -5.527       7.101
C(Position)[T.18]                     3.6434      2.839      1.283      0.200      -1.938       9.224
C(Position)[T.19]                    -0.0199      3.786     -0.005      0.996      -7.461       7.421
C(Position)[T.20]                     4.3164      2.839      1.520      0.129      -1.265       9.897
C(Position)[T.21]                     1.9278      2.932      0.657      0.511      -3.836       7.692
C(Position)[T.22]                     5.1128      3.052      1.675      0.095      -0.887      11.112
C(Position)[T.23]                     3.3980      3.212      1.058      0.291      -2.916       9.712
C(Position)[T.24]                    10.3074      3.212      3.209      0.001       3.993      16.622
C(Position)[T.25]                     5.2631      2.839      1.854      0.064      -0.318      10.844
C(Position)[T.26]                     8.9413      4.399      2.033      0.043       0.295      17.587
C(Position)[T.27]      

In [51]:
res = sm.stats.anova_lm(model, typ=2)
res

,sum_sq,df,F,PR(>F)
C(Position),4252.113371,29.0,4.871783,5.441536e-14
C(Texture),14.745456,2.0,0.244968,7.828411e-01
C(Position):C(Texture),1523.257269,58.0,0.872622,7.339915e-01
Residual,12670.711917,421.0,NaN,NaN


In [25]:
# if interaction not sig
model2 = ols('FiringRate ~ C(Position)+C(Texture)', df).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             FiringRate   R-squared:                       0.231
Model:                            OLS   Adj. R-squared:                  0.181
Method:                 Least Squares   F-statistic:                     4.630
Date:                Mon, 15 Apr 2019   Prob (F-statistic):           5.96e-14
Time:                        16:40:33   Log-Likelihood:                -1571.8
No. Observations:                 510   AIC:                             3208.
Df Residuals:                     478   BIC:                             3343.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.2589      1.356      0.191      0.849      -2.406       2.924
C(Position)[T.1]      0.7343      2.110      0.348      0.728      -3.411       4.880
C(Position)[T.2]      0.7991      1.820      0.439      0.661      -2.776       4.374
C(Position)[T.3]      1.1028      1.967      0.561      0.575      -2.762       4.967
C(Position)[T.4]      2.1473      1.800      1.193      0.233      -1.389       5.684
C(Position)[T.5]     -0.0516      1.898     -0.027      0.978      -3.781       3.678
C(Position)[T.6]      1.3816      1.762      0.784      0.433      -2.081       4.844
C(Position)[T.7]      0.8433      1.779      0.474      0.636      -2.652       4.339
C(Position)[T.8]      1.1424      1.800      0.635      0.526      -2.395       4.680
C(Position)[T.9]      0.7999      1.714      0.467      0.641      -2.567       4.167
C(Position)[T.10]     0.7899      1.688      0.468      0.640      -2.527       4.106
C(Position)[T.11]     0.2521      1.779      0.142      0.887      -3.243       3.747
C(Position)[T.12]     0.5986      1.744      0.343      0.732      -2.828       4.025
C(Position)[T.13]     0.3850      1.819      0.212      0.833      -3.190       3.960
C(Position)[T.14]     1.7616      1.822      0.967      0.334      -1.818       5.342
C(Position)[T.15]     0.5305      1.779      0.298      0.766      -2.965       4.026
C(Position)[T.16]     4.0803      1.899      2.149      0.032       0.350       7.811
C(Position)[T.17]     1.0547      1.821      0.579      0.563      -2.523       4.632
C(Position)[T.18]     3.0307      1.899      1.596      0.111      -0.700       6.761
C(Position)[T.19]     1.6206      1.847      0.877      0.381      -2.009       5.250
C(Position)[T.20]     4.5173      1.843      2.451      0.015       0.896       8.139
C(Position)[T.21]     3.4476      1.931      1.785      0.075      -0.346       7.242
C(Position)[T.22]     6.6015      1.821      3.626      0.000       3.024      10.179
C(Position)[T.23]     4.9824      1.968      2.532      0.012       1.116       8.849
C(Position)[T.24]    11.9756      1.932      6.198      0.000       8.179      15.772
C(Position)[T.25]     7.8528      1.869      4.201      0.000       4.180      11.526
C(Position)[T.26]     9.8486      2.175      4.529      0.000       5.576      14.121
C(Position)[T.27]     8.1972      2.173      3.773      0.000       3.928      12.466
C(Position)[T.28]     3.4127      2.590      1.317      0.188      -1.677       8.503
C(Position)[T.29]     1.4044      4.079      0.344      0.731      -6.610       9.419
C(Texture)[T.B]       0.1172      0.616      0.190      0.849      -1.093       1.327
C(Texture)[T.C]       0.3938      0.584      0.674      0.501      -0.754       1.542
=========================================================================

In [26]:
res2 = sm.stats.anova_lm(model2, typ=2)
res2

,sum_sq,df,F,PR(>F)
C(Position),4258.302050,29.0,4.946183,1.489087e-14
C(Texture),14.274171,2.0,0.240410,7.864004e-01
Residual,14190.449774,478.0,NaN,NaN


In [11]:
def run_ANOVA(alley):
    """ Creates OLS model and generates ANOVA
    table for a given alley. Factors are position
    with levels == bins, and textures with levels == A,B,C
    Initially test with interaction and if not significant
    rerun without interaction. Return model and anova table.
    Unit must be in local namespace"""
    posLevels, txtLevels, firingRates = [], [], []

    for visit in unit.alleys[alley]:
        frs = visit['ratemap1d']
        txt = visit['metadata']['stimulus']

        firingRates.extend(frs)
        posLevels.extend(list(range(nbins)))
        txtLevels.extend([txt]*nbins)
        
    dataDict = {'Position': posLevels, 'Texture': txtLevels, 'FiringRate': firingRates}
    df = pandas.DataFrame(dataDict, columns = ['Position', 'Texture', 'FiringRate'])
    
    model = ols('FiringRate ~ C(Position)*C(Texture)', df).fit()
    res = sm.stats.anova_lm(model, typ=2)
    
    if  res['PR(>F)']['C(Position):C(Texture)'] > alphaCorr:
        model = ols('FiringRate ~ C(Position)+C(Texture)', df).fit()
        res = sm.stats.anova_lm(model, typ=2)
        
    return model, res
        
    

In [15]:
beltwayAlleyList = [16, 17, 3, 1, 5, 7, 8, 10, 11]
beltwaynamelookup = {16:1, 17:2, 3:3, 1:4, 5:5, 7:6, 8:7, 10:8, 11:9}
datapath = "E:\\Ratterdam\\R781\\"
file = open(datapath + f"{expCode}_ANOVA_Results_{alphaCorr}_justTxtMainEffect.csv","w")


clustResults = {c:[] for c in clustList}

for clust in clustList:
    
    anova_results = {a:{'Position':-1, 'Texture':-1, 'PositionXTexture':-1} for a in beltwayAlleyList}
    
    file.write(f"Unit {clust}")
    file.write('\n')
    
    print(f"{clust}")
    
    unit = Core.UnitData(clust, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
    unit.loadData_raw()
    
    for alley in beltwayAlleyList:

        try:
            model, res = run_ANOVA(alley)
            success = True
        except:
            success = False

        #scrape results
        if success:
            anova_results[alley]['Position'] = res['PR(>F)']['C(Position)']
            anova_results[alley]['Texture'] = res['PR(>F)']['C(Texture)']

            if 'C(Position):C(Texture)' in res['PR(>F)']:
                anova_results[alley]['PositionXTexture'] = res['PR(>F)']['C(Position):C(Texture)'] 

    # write to csv
    for alley in beltwayAlleyList:
        for data in ['Texture']:
            if anova_results[alley][data] > 0 and anova_results[alley][data] < alphaCorr:
                clustResults[clust].append(beltwaynamelookup[alley])

    file.write(str(clustResults[clust]))
    file.write('\n')



    for alley in beltwayAlleyList:    
        file.write(f"Alley {beltwaynamelookup[alley]}")
        file.write('\n')
        for data in ['Position', 'Texture', 'PositionXTexture']:
            file.write(f"{data} p-value: ")
            file.write(str(anova_results[alley][data]))
            file.write('\n')
        file.write('\n')

    file.write('\n')
        
file.write("Summary")
file.write('\n')

total = 0
for clust in clustList:
    file.write(clust)
    file.write('\n')
    file.write(str(clustResults[clust]))
    file.write('\n')
    file.write('\n')
    
    if len(clustResults[clust]) > 0:
        total += 1
        
file.write(f"Total cells with a texture effect: {total}")




file.close()

TT11\cl-maze1.1


C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:138: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:138: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:144: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:144: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:310: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:306: RuntimeWarning: invalid value encountered

TT11\cl-maze1.2
TT3\cl-maze1.1
TT3\cl-maze1.2
TT3\cl-maze1.3
TT3\cl-maze1.4
TT3\cl-maze1.5
TT3\cl-maze1.6
TT3\cl-maze1.7
TT5\cl-maze1.1
TT5\cl-maze1.2
TT5\cl-maze1.3
TT5\cl-maze1.4
TT6\cl-maze1.1
TT6\cl-maze1.2
TT6\cl-maze1.3
TT6\cl-maze1.4
TT6\cl-maze1.5
TT6\cl-maze1.6
TT6\cl-maze1.7
TT6\cl-maze1.8
TT6\cl-maze1.9
TT9\cl-maze1.1
TT9\cl-maze1.2
TT9\cl-maze1.3
TT9\cl-maze1.4
TT9\cl-maze1.5
TT9\cl-maze1.6
TT9\cl-maze1.7


## Revisiting two-factor anova for beltway remapping data
### late September 2020

In [154]:
rat = 'R781'
expCode = 'BRD3'
datafile = f'E:\\Ratterdam\\{rat}\\{rat}{expCode}\\'
savepath = f'E:\\Ratterdam\\{rat}\\anovas\\'

alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
clustlist = util.getClustList(datafile)
population = {}

for clust in clustlist:
    unit = Core.UnitData(clust, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
    unit.loadData_raw()
    validalleys = []
    valid, acorr, alleys = checkInclusion(unit, 1)
    if valid:
        print(clust)
        unit.acorr = acorr
        unit.validAlleys = alleys
        population[clust] = unit


E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:174: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:174: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:180: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:180: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


TT11\cl-maze1.1
TT11\cl-maze1.2
TT12\cl-maze1.1
TT12\cl-maze1.2
TT3\cl-maze1.3
TT3\cl-maze1.4
TT3\cl-maze1.5
TT3\cl-maze1.6
TT3\cl-maze1.7
TT3\cl-maze1.8
TT5\cl-maze1.1
TT5\cl-maze1.3
TT6\cl-maze1.1
TT6\cl-maze1.2
TT6\cl-maze1.3
TT6\cl-maze1.4
TT6\cl-maze1.6
TT6\cl-maze1.8
TT6\cl-maze1.9
TT9\cl-maze1.1
TT9\cl-maze1.2
TT9\cl-maze1.5
TT9\cl-maze1.6
TT9\cl-maze1.7


In [383]:
# do for a day
print(rat)
print(expCode)
print(f"# cells included: {len(population.keys())}")
allres = []
stamp = util.genTimestamp()

with PdfPages(savepath+stamp+f"{unit.name}_{expCode}_smooth{Def.smoothing_1d_sigma}_{Def.singleAlleyBins[0]-1}bins_{Def.velocity_filter_thresh}vfilt_R{Def.includeRewards}_ANOVAHists"+".pdf") as pdf:

    for clust in population.keys():
        print(clust)
        include, alphaCorr, alleys = checkInclusion(population[clust])
        if include is True:
            for alley in alleys:
                #print(alley)
                res, inputdata = run_ANOVA(population[clust], alley, alphaCorr)
                allres.append(res)

                fig, ax = plt.subplots(2,2,figsize=(8,6))
                for i,(title,data) in enumerate(zip(['All','A','B','C'],inputdata[:-2])): # data = [firngRates,a,b,c,shapiroP, bartlettP]
                    fig.axes[i].hist(data,bins=15)
                    fig.axes[i].set_title(title)
                plt.suptitle(f"{population[clust].name} Alley {alley} Log Transform Infield Data. Shapiro p = {round(inputdata[-2],6)}, Bartlett p = {round(inputdata[-1],6)}",fontsize=8)
                pdf.savefig()
                plt.close()
            
            
allres = pd.concat(allres)
allres.to_csv(savepath+f"{stamp}_{rat}_{expCode}_{Def.velocity_filter_thresh}vfilt_{Def.includeRewards}R_ANOVAresults.csv")

# Create summary file. Parameters of the ANOVAs, what(if any) units pass, with what effect, etc
mainpassing = [[allres['name'][i],allres['Shapiro P'][i],allres['Bartlett P'][i]] for i in range(allres.shape[0]) if allres.index.values[i]=='C(Texture)' and allres['PR(>F)'][i]<allres['adjP'][i]]
interactionpassing = [[allres['name'][i],allres['Shapiro P'][i],allres['Bartlett P'][i]] for i in range(allres.shape[0]) if allres.index.values[i]=='C(Position):C(Texture)' and allres['PR(>F)'][i]<allres['adjP'][i]]

nmaincells = len(set([p[0].split("_")[0] for p in mainpassing]))
nintcells = len(set([p[0].split("_")[0] for p in interactionpassing]))
ncells= len(set([p[0].split("_")[0] for p in mainpassing] + [p[0].split("_")[0] for p in interactionpassing])) # set of all cells
p = binom_test(ncells, len(population), 0.05, 'greater')
pmain = binom_test(nmaincells, len(population), 0.05, 'greater')
pint = binom_test(nintcells, len(population), 0.05, 'greater')

with open(savepath+f"{stamp}_{rat}_{expCode}_{Def.velocity_filter_thresh}vfilt_{Def.includeRewards}R_ANOVAsummary.csv","w",newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow([f"{stamp} {rat} {expCode}"])
    writer.writerow([f"Two-factor Type II/III ANOVA (Type II if no interaction under type III). Factors: Texture with levels [A,B,C] and Spatial Bins with levels 0-{Def.singleAlleyBins[0]-2}."])
    writer.writerow(["Testing main effect of texture"])
    writer.writerow([f"{ncells} units pass / {len(population)} units. {round(ncells/len(population),2)*100}%, \
                    one-sided binomial test alpha=0.05 p-value = {p}"])
    writer.writerow([f"Units/alleys with main effect of texture ({nmaincells}/{len(population)},{round(nmaincells/len(population),3)},{round(pmain,6)})","Shapiro P", "Bartlett P"])
    for data in mainpassing:
        writer.writerow(data)
    writer.writerow([f"Units/alleys with interaction effect ({nintcells}/{len(population)},{round(nintcells/len(population),3)},{round(pint,6)})", "Shapiro P", "Bartlett P"])
    for data in interactionpassing:
        writer.writerow(data)

R886
BRD2
# cells included: 21
TT11\cl-maze1.1
TT11\cl-maze1.2
TT11\cl-maze1.3
TT11\cl-maze1.4
TT11\cl-maze1.6
TT11\cl-maze1.7
TT11\cl-maze1.8
TT12\cl-maze1.1
TT12\cl-maze1.10
TT12\cl-maze1.11
TT12\cl-maze1.2
TT12\cl-maze1.3
TT12\cl-maze1.6
TT12\cl-maze1.7
TT12\cl-maze1.9
TT2\cl-maze1.2
TT2\cl-maze1.4
TT5\cl-maze1.1
TT5\cl-maze1.4
TT9\cl-maze1.1
TT9\cl-maze1.2


In [366]:
def run_ANOVA(unit,alley,alphaCorr):
    """ Creates OLS model and generates ANOVA
    table for a given alley. Factors are position
    with levels == bins, and textures with levels == A,B,C
    Initially test with interaction and if not significant
    rerun without interaction. Return model and anova table.
    """
    posLevels, txtLevels, firingRates = [], [], []
    
    _, field = findField(unit,alley) # first return is bool if field is detected. Dont need to check bc this fx wont be run otherwise

    nbins = field.shape[0]
    for visit in unit.alleys[alley]:
        frs = visit['ratemap1d'][field]
        txt = visit['metadata']['stimulus']
        firingRates.extend(frs)
        posLevels.extend(list(range(nbins)))
        txtLevels.extend([txt]*nbins)

    firingRates, posLevels, txtLevels = np.asarray(firingRates), np.asarray(posLevels), np.asarray(txtLevels)
    valid = ~np.isnan(firingRates)
    txtLevels, posLevels, firingRates = txtLevels[valid], posLevels[valid], firingRates[valid]
    a,b,c = firingRates[np.where(txtLevels=='A')[0]], firingRates[np.where(txtLevels=='B')[0]], firingRates[np.where(txtLevels=='C')[0]]
    #nonzeroIdx = np.where(firingRates>0)[0]
    #firingRates, txtLevels, posLevels = np.log(firingRates[nonzeroIdx]), txtLevels[nonzeroIdx], posLevels[nonzeroIdx]
    firingRates = np.log(firingRates+1)
    a, b, c = np.log(a+1), np.log(b+1), np.log(c+1)

    dataDict = {'Position': posLevels, 'Texture': txtLevels, 'FiringRate': firingRates}
    df = pandas.DataFrame(dataDict, columns = ['Position', 'Texture', 'FiringRate'])

    model = ols('FiringRate ~ C(Position)*C(Texture)', df).fit()
    res = sm.stats.anova_lm(model, typ=3)

    if  res['PR(>F)']['C(Position):C(Texture)'] >= alphaCorr: #ie if not sig
        #model = ols('FiringRate ~ C(Position)+C(Texture)', df).fit()
        model = ols('FiringRate ~ C(Position)*C(Texture)', df).fit()
        res = sm.stats.anova_lm(model, typ=2) # type II is more poweful if no interaction
        alphaCorr = alphaCorr/2 # if we run a second anova test take that into account wrt pvalue

    res['name'] = f"{unit.name}_{alley}" 
    res['adjP'] = alphaCorr

    #run tests of ANOVA assumptions and add results as columns
    _, shapiroP = shapiro(firingRates)
    _, bartlettP = bartlett(a,b,c)
    res['Shapiro P'] = shapiroP
    res['Bartlett P'] = bartlettP

    return res, [firingRates, a, b, c, shapiroP, bartlettP]

In [149]:
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

def findField(unit,alley,sthresh=3,rthresh=0.5,pctThresh=None):
    """
    Identify a field as a set of sthresh or more contiguous bins
    greater than some thresh
    rthresh - an absolute thresh in Hz
    pct thresh - a pct of max 
    One of these must be None, cant have both
    """
    rms = np.empty((0, Def.singleAlleyBins[0]-1))
    for visit in unit.alleys[alley]:
        rm = visit['ratemap1d']
        rms = np.vstack((rms, rm))
        
    if rthresh is not None and pctThresh is not None:
        print("Error - conflicting thresh definitions")
    mean = np.nanmean(rms, axis=0)
    if rthresh is not None:
        thresh = rthresh        
        fi = np.where(mean>=rthresh)[0]
    elif pctThresh is not None:
        thresh = pctThresh
        fi = np.where(mean>=(pctthresh*np.nanmax(mean)))[0]        
    
    field = True
    try:
        field_idx = np.concatenate(([i for i in consecutive(fi) if len(i)>=sthresh]))
    except:
        field = False
        field_idx = None
    return field, field_idx

In [153]:
def checkInclusion(unit,ncompsperalley):
    """
    Apply inclusion criteria to a unit, deciding which(if any)
    alleys will be included in analysis. If 0, cell is not used.
    return: inclusion bool, adj alpha, alley(s) to be included
    adj alpha is 0.05 / (# alleys included * # comparisons per alley (arg: ncompsperalley))
    """
    validalleys = []
    for alley in Def.beltwayAlleys:
        passesCheck = Filt.checkMinimumPassesActivity(unit, alley, pass_thresh=12)
        fieldCheck, _ = findField(unit, alley)
        if passesCheck is True and fieldCheck is True:
            validalleys.append(alley)
    if len(validalleys)>0:
        alphaCorr = 0.05/(len(validalleys)*ncompsperalley)
        include = True
    else:
        alphaCorr = None
        include = False
    return include, alphaCorr, validalleys

In [447]:
unit = population['TT12\\cl-maze1.7']
alley=16
txtLevels, firingRates = [], []

_, field = findField(unit,alley) # first return is bool if field is detected. Dont need to check bc this fx wont be run otherwise

nbins = field.shape[0]
for visit in unit.alleys[alley]:
    frs = visit['ratemap1d'][field]
    txt = visit['metadata']['stimulus']
    firingRates.append(np.nanmean(frs))
    txtLevels.append(txt)

firingRates, txtLevels = np.asarray(firingRates), np.asarray(txtLevels)
valid = ~np.isnan(firingRates)
txtLevels, firingRates = txtLevels[valid],  firingRates[valid]
a,b,c = firingRates[np.where(txtLevels=='A')[0]], firingRates[np.where(txtLevels=='B')[0]], firingRates[np.where(txtLevels=='C')[0]]

firingRates = np.log(firingRates+1)
a, b, c = np.log(a+1), np.log(b+1), np.log(c+1)

### One way Kruskal-Wallis test 

In [451]:
rat = 'R859'
expCode = 'BRD5'
datafile = f'E:\\Ratterdam\\{rat}\\{rat}{expCode}\\'
savepath = f'E:\\Ratterdam\\{rat}\\anovas\\'

alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
clustlist = util.getClustList(datafile)
population = {}

for clust in clustlist:
    unit = Core.UnitData(clust, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
    unit.loadData_raw()
    include, alphaCorr, alleys = checkInclusion(unit)
    if include is True:      
        print(clust)
        population[clust] = unit

E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:174: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:174: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:180: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:180: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


TT1\cl-maze1.1
TT1\cl-maze1.2
TT1\cl-maze1.3
TT1\cl-maze1.4
TT1\cl-maze1.5
TT1\cl-maze1.6
TT10\cl-maze1.10
TT10\cl-maze1.11
TT10\cl-maze1.12
TT10\cl-maze1.6
TT10\cl-maze1.7
TT13\cl-maze1.1
TT13\cl-maze1.2
TT13\cl-maze1.3
TT13\cl-maze1.4
TT13\cl-maze1.7
TT13\cl-maze1.8
TT6\cl-maze1.10
TT6\cl-maze1.11
TT6\cl-maze1.12
TT6\cl-maze1.14
TT6\cl-maze1.15
TT6\cl-maze1.16
TT6\cl-maze1.2
TT6\cl-maze1.4
TT6\cl-maze1.6
TT6\cl-maze1.7
TT6\cl-maze1.8
TT6\cl-maze1.9


In [454]:
for clust in population.keys():
    unit = population[clust]
    include, alphaCorr, alleys = checkInclusion(unit)
    print("------------------------------")
    print(clust)
    print(f"Adjusted p = {alphaCorr}")
    for alley in alleys:
        print("=======")
        print(alley)
        
        txtLevels, firingRates = [], []
        _, field = findField(unit,alley) # first return is bool if field is detected. Dont need to check bc this fx wont be run otherwise
        nbins = field.shape[0]
        
        for visit in unit.alleys[alley]:
            frs = visit['ratemap1d'][field]
            txt = visit['metadata']['stimulus']
            firingRates.append(np.nanmax(frs))
            txtLevels.append(txt)

        firingRates, txtLevels = np.asarray(firingRates), np.asarray(txtLevels)
        valid = ~np.isnan(firingRates)
        txtLevels, firingRates = txtLevels[valid],  firingRates[valid]
        a,b,c = firingRates[np.where(txtLevels=='A')[0]], firingRates[np.where(txtLevels=='B')[0]], firingRates[np.where(txtLevels=='C')[0]]
        stat, p = kruskal(a,b,c)
        print(f"Kruskal-Wallis p = {p}")

------------------------------
TT1\cl-maze1.1
Adjusted p = 0.005555555555555556
16
Kruskal-Wallis p = 0.3225780177787122
17
Kruskal-Wallis p = 0.3491727295324705
3
Kruskal-Wallis p = 0.6502987385026047
1
Kruskal-Wallis p = 0.4646973671070762
5
Kruskal-Wallis p = 0.7157294120056974
7
Kruskal-Wallis p = 0.7549802556970981
8
Kruskal-Wallis p = 0.9862071167439183
10
Kruskal-Wallis p = 0.7096237306605161
11
Kruskal-Wallis p = 0.48025259813256105
------------------------------
TT1\cl-maze1.2
Adjusted p = 0.008333333333333333
16
Kruskal-Wallis p = 0.06308017543623266
3
Kruskal-Wallis p = 0.48039502342957985
1
Kruskal-Wallis p = 0.11327966960167336
7
Kruskal-Wallis p = 0.5191809308338287
10
Kruskal-Wallis p = 0.4305045903523015
11
Kruskal-Wallis p = 0.7582944082652724
------------------------------
TT1\cl-maze1.3
Adjusted p = 0.016666666666666666
16
Kruskal-Wallis p = 0.5908054727996033
7
Kruskal-Wallis p = 0.5808108795272395
8
Kruskal-Wallis p = 0.2685336191329722
----------------------------

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: All-NaN slice encountered


## Wilcoxon signed rank test

In [272]:
rat = 'R886'
expCode = 'BRD2'
datafile = f'E:\\Ratterdam\\{rat}\\{rat}{expCode}\\'
savepath = f'E:\\Ratterdam\\{rat}\\anovas\\'

alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
clustlist = util.getClustList(datafile)
population = {}

for clust in clustlist:
    unit = Core.UnitData(clust, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
    unit.loadData_raw()
    validalleys = []
    valid, acorr, alleys = checkInclusion(unit, 3)
    if valid:
        print(clust)
        unit.acorr = acorr
        unit.validAlleys = alleys
        population[clust] = unit

E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:174: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:174: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:180: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:180: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


TT11\cl-maze1.1
TT11\cl-maze1.2
TT11\cl-maze1.3
TT11\cl-maze1.4
TT11\cl-maze1.6
TT11\cl-maze1.7
TT11\cl-maze1.8
TT12\cl-maze1.1
TT12\cl-maze1.10
TT12\cl-maze1.11
TT12\cl-maze1.2
TT12\cl-maze1.3
TT12\cl-maze1.6
TT12\cl-maze1.7
TT12\cl-maze1.9
TT2\cl-maze1.2
TT2\cl-maze1.4
TT5\cl-maze1.1
TT5\cl-maze1.4
TT9\cl-maze1.1
TT9\cl-maze1.2


In [273]:
print(f"{rat}, {expCode} {util.genTimestamp()}")
passcounts = 0
passingeffects = []
nonpassingeffects = []
for unitname in population.keys():
    print(unitname)
    unit = population[unitname]
    passbool = False
    for alley in unit.validAlleys:
        print(alley)
        visits, txts = [],[]
        for visit in unit.alleys[alley]:
            visits.append(visit['ratemap1d'])
            txts.append(visit['metadata']['stimulus'])
        visits = np.asarray(visits)
        txts = np.asarray(txts)
        a,b,c = np.nanmean(visits[np.where(txts=='A')[0],:],axis=0),\
                np.nanmean(visits[np.where(txts=='B')[0],:],axis=0),\
                np.nanmean(visits[np.where(txts=='C')[0],:],axis=0)
        ab, bc, ca = scipy.stats.wilcoxon(a,b), scipy.stats.wilcoxon(b,c), scipy.stats.wilcoxon(c,a)
        
        if ab[1] < unit.acorr:
            print("AB")
            passingeffects.append(t2r(ab[0], Def.singleAlleyBins[0]-1))
            passbool = True
        else:
             nonpassingeffects.append(t2r(ab[0], Def.singleAlleyBins[0]-1))
            
        if bc[1] < unit.acorr:
            print("BC")
            passingeffects.append(t2r(bc[0], Def.singleAlleyBins[0]-1))
            passbool = True
        else:
            nonpassingeffects.append(t2r(bc[0], Def.singleAlleyBins[0]-1))
            
        if ca[1] < unit.acorr:
            print("CA")
            passingeffects.append(t2r(ca[0], Def.singleAlleyBins[0]-1))
            passbool = True
        else:
            nonpassingeffects.append(t2r(ca[0], Def.singleAlleyBins[0]-1))
            
    if passbool == True:
        passcounts += 1
        
                                
passingeffects, nonpassingeffects = np.asarray(passingeffects), np.asarray(nonpassingeffects)
print(f"{passcounts}/{len(population)} ({round(passcounts/len(population),3)}) units have at least one alley passing test.")
print(scipy.stats.binom_test(passcounts, len(population), 0.05, 'greater'))

R886, BRD2 2020107-151022
TT11\cl-maze1.1
16
17
TT11\cl-maze1.2
16
AB
17
TT11\cl-maze1.3
16
AB
TT11\cl-maze1.4
16
AB
8
10
TT11\cl-maze1.6
16
TT11\cl-maze1.7
16
17
TT11\cl-maze1.8
16
AB
BC
TT12\cl-maze1.1
16
TT12\cl-maze1.10
16
17
3
1
5
7
8
10
11
TT12\cl-maze1.11
16
7
CA
8
TT12\cl-maze1.2
16
TT12\cl-maze1.3
16
11
TT12\cl-maze1.6
16
AB
CA
17
1
11
TT12\cl-maze1.7
16
17
AB
7
8
TT12\cl-maze1.9
16
10
11
TT2\cl-maze1.2
7
TT2\cl-maze1.4
16
5
7
TT5\cl-maze1.1
11
AB
BC
TT5\cl-maze1.4
1
8
10
TT9\cl-maze1.1
16
17
3
AB
5
7
TT9\cl-maze1.2
1
5
7
8
9/21 (0.429) units have at least one alley passing test.
3.3088138943127526e-07


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [274]:
plt.violinplot([passingeffects, nonpassingeffects])
plt.ylabel("Effect size",fontsize=18)
plt.title(f"{rat} {expCode} Effect size R",fontsize=22)
plt.xticks([1,2], ["Passing", "Nonpassing"], fontsize=18)

([<matplotlib.axis.XTick at 0x24235409cc0>,
 <a list of 2 Text xticklabel objects>)

In [191]:
def sumtorial(x):
    return ((x**2) + x )/2

def t2r(t,n):
    """ Calc effectsize r from Wilcoxon signed rank test stat. From Kirby 2014"""
    rank = sumtorial(n)
    rankA = t
    rankB = rank - t
    return abs((rankA/rank) - (rankB/rank))